### 이 노트북은 Hands-on Machine Learning with Scikit-Learn and TensorFlow 책의 2 장 [GitHub에 있는  실습코드](https://github.com/ageron/handson-ml)  에 기반한 것입니다.   추가 설명 및 코드로 원 실습코드와 내용이 다릅니다.
<br>
 

**Chapter 2 – End-to-end Machine Learning project (머신러닝, 데이터분석 프로젝트 훑어보기)**
- 데이터 : 1990년 캘리포니아 주택들에 관한 인구조사 데이터  
- 머신러닝 시스템의 목표 : 어떤 지역(District)에 위치한 집들의 중간값 예측 
- 사용 목적 : 예측치를 다른 정보들과 결합해 투자 결정 


**Note**: You may find little differences between the code outputs in the book and in these Jupyter notebooks: these slight differences are mostly due to the random nature of many training algorithms: although I have tried to make these notebooks' outputs as constant as possible, it is impossible to guarantee that they will produce the exact same output on every platform. Also, some data structures (such as dictionaries) do not preserve the item order. Finally, I fixed a few minor bugs (I added notes next to the concerned cells) which lead to slightly different results, without changing the ideas presented in the book.

# Setup

First, let's make sure this notebook works well in both python 2 and 3, import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures:

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)    # 랜덤함수 초기화 

# To plot pretty figures
%matplotlib inline     
import matplotlib
import matplotlib.pyplot as plt   # 시각화 패키지 
plt.rcParams['axes.labelsize'] = 14   # 시각화 관련 세팅 
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."   # 현재 디렉토리를 가르킴 
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):  # 그림 저장 함수 
    if not os.path.isdir(IMAGES_PATH):     
        os.makedirs(IMAGES_PATH)
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

# Get the data
- 보통, 데이터 분석은 외부에 있는 데이터를 프로그램 내부로 갖고 오는 것으로부터 시작됨 

In [5]:
import pandas as pd

data_path = "../input/housing.csv"
housing = pd.read_csv(data_path)

# see the basic info
housing.info()

check null values 

In [6]:
np.sum(pd.isnull(housing)) 

housing이 pandas의 DataFrame 타입 (객체) 임을 알 수 있음.  인스턴스 (example, row)의 갯수가 20640 이며, 10개의 column들 (9개는 실수형, ocean_proximity는 object 타입(대개 이는 string을 뜻함)), 결측치가 total_bedrooms 열에 207개 있음.   
<br>
DataFrame의 행과 열을 여러가지 명칭으로 부름. 행이 보통 하나의 데이터 포인트를 나타내기에 이를 instance, example, observation, record, row 로 말함.  또, 열은 데이터의 특징을 보통 나타내기에 이를 feature, attribute, field, column이라 함.  또 feature들로 response를 예측하기에 feature를 변수, predictor라 하기도 함.    

그런데, 주의할 점은 열 중에서 어떤 것은 response로 사용할 수 있으므로 모든 열이 입력변수/predictor가 됨은 아니다.

In [7]:
housing["ocean_proximity"].value_counts()  # pandas 인덱싱의 한 예.  여기서는 ocean_proximity 열을 선택 

위의 작업은 왜 했을까?  housing의 ocean_proximity 열은 다른 열과 달리 string 으로 되어있음. 즉, 이 열(변수)는 qualitative 변수.  변수명을 보니 ocean_proximity, 즉 "바다 근접성"를 나타내고 있음.  바다와의 거리를 몇 km 같이 숫자로 나타낼 수 있지만 여기서는 문자열로 나타내었음.  궁금한 것은 바다 근접성을 "몇 가지 부류로 나타내었나" 이었음. 살펴보니 5가지 부류로 나타냄.  가장 많은 부류가 "1H OCEAN" 으로 전체 20640 중 9136 이 이에 해당. 5개는 섬에 있는 집인 모양.

In [8]:
housing.describe(include='all')

### 이 즈음에서 우리가 행하려는 머신러닝 데이터 분석은 어떤 형태이고, 어떻게 진행함이 맞을까를 생각해 봐야 함
- 우리가 예측하려는 response는 district의 주택들의 중간값 (median value)  
- 데이터를 보니 중간값 정보가 있음 (median_house_value). 따라서 우리 분석 문제는 ***Supervised Learning***으로 할 수 있겠음.  그리고, median_house_value 값이 숫자이니 ***regression*** 타입 문제. 정확히는 여러개의 변수(predictor, columns)들을 사용할 것이니 다변수 regression (multivariate regression) 문제 
- 결측치도 별로 없고, 데이터가 예쁘고, 이상한 데이터 값 잘 안보이고, 이 정도면 무지 분석하기 좋은 상태 
- 데이터는 다 준비되었고, 사이즈 크지 않으니 단순히 batch learning 스타일로 하면 좋겠음.  
- 무지 쉬운 문제로 보임

** 탐색적 분석**을 통해 데이터 특성을 좀 더 살피자
- Histogram : shows the number of instances (on the vertical axis) that have a given
value range (on the horizontal axis)

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))   # pandas DataFrame이 간단한 시각화 함수를 갖고 있음 
save_fig("attribute_histogram_plots")
plt.show()

히스토그램을 보니 housing_median_age와 median_house_value의 상한 값이 있다. median_house_value 경우 상한 값이 500,000 이다.  그런데, median_house_value 이 우리가 예측하려는 target (response) 이다.  이 데이터를 그대로 사용하면, 우리 모델은 median_house_value 이 500,000 이 넘을 경우를 학습을 못하게 된다.  

데이터를 신경써 보니 문제가 될 것들이 보임
- median_house_value 상한이 존재함에 따른 문제
- attributes have very different scales (어떤 것은 0~50 사이, 어떤 것은 0~40000) 
- tail heavy (정규 분포 모양이 아님) 

In [10]:
housing.info()

In [11]:
# to make this notebook's output identical at every run
np.random.seed(42)

#### 우리가 갖고 있는 20640 개의 데이터 인스턴스들을 train set과 test set으로 나눔.  학습 모델을 훈련시킬 때에 train set을 이용.  

**우리가 곧 배울 [Scikit-Learn](http://scikit-learn.org/stable/index.html)에는 데이터를 train/test set으로 나누는 함수가 있다.  그리고 많은 머신러닝 및 여러 함수들이 있다.  **

In [12]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [13]:
train_set.shape, test_set.shape

In [14]:
test_set.head()

In [15]:
housing["median_income"].hist()

### Stratified Sampling 및 Split

In [16]:
# Divide by 1.5 to limit the number of income categories
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
# Label those above 5 as 5
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

In [17]:
housing["income_cat"].value_counts()

In [18]:
housing["income_cat"].hist()

In [19]:
# StratifiedShuffleSplit 이용 
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [20]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [21]:
housing["income_cat"].value_counts() / len(housing)

In [22]:
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()
compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [23]:
compare_props

### Feature 들 간의 상관 관계

In [25]:
corr_matrix = housing.corr()

In [26]:
corr_matrix

In [27]:
corr_matrix["median_house_value"].sort_values(ascending=False)

median_house_value 와 median_income 은 상당히 강한 양적 상관 관계가 있다 : 같이 증가하고, 같이 감소한다.

In [28]:
from pandas.tools.plotting import scatter_matrix # For older versions of Pandas
# from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
save_fig("scatter_matrix_plot")

In [29]:
housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1, c='blue')
plt.axis([0, 16, 0, 550000])   # x축 범위: 0~16,  y축 범위: 0~550000 
save_fig("income_vs_house_value_scatterplot")

**기존 변수를 이용해 더 의미있는 새로운 변수를 만들어 봄 - Create new features **

In [30]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [31]:
housing.info()

In [32]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

새로운 변수(predictor) bedrooms_per_room를 만들어 median_house_value와의 상관관계를 보니 퍽 의미있는 음의 상관관계를 보임. 즉, 전체 방 개수 중 침실의 비율이 높으면 median_house_value 가 감소함. 다시 말해, 전체 방 중 침실의 수가 적으면 주택 값이 높아지는 경향을 보임.  **침실말고 다른 방들이 많은 주택들이 밀집한 동네들의 집값이 높음.**

In [33]:
housing.plot(kind="scatter", x="rooms_per_household", y="median_house_value",
             alpha=0.2)
plt.axis([0, 5, 0, 520000])
plt.show()

In [34]:
housing.describe()

# 추가적 전처리 - More feature tuning

In [36]:
housing.info()

<h3>Check skewness for numeric data

In [37]:
from scipy import stats
from scipy.stats import norm, skew 

In [38]:
numeric_features = housing.dtypes[housing.dtypes != "object"].index
numeric_features

In [39]:
# Check the skew of all numerical features
skewed_feats = housing[numeric_features].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness

<h5>deal with "rooms per household" /  "rooms per household"부터 log처리 및 범위제한

In [40]:
import seaborn as sns

In [41]:
sns.distplot(housing['rooms_per_household']);

In [42]:
housing["rooms_per_household"] = np.log1p(housing["rooms_per_household"])
sns.distplot(housing['rooms_per_household']);

<h5>deal with "population_per_household"

In [43]:
sns.distplot(housing['population_per_household']);

In [44]:
housing["population_per_household"] = np.log1p(housing["population_per_household"])
sns.distplot(housing['population_per_household']);

In [45]:

housing["population_per_household"].where(housing["population_per_household"] < 2.75, 2.75, inplace=True)

In [46]:
sns.distplot(housing['population_per_household']);

In [47]:
#Check the skew of all numerical features
skewed_feats = housing[numeric_features].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness

<h5>deal with "population"

In [48]:
sns.distplot(housing['population']);


In [49]:
housing["population"] = np.log1p(housing["population"])

In [50]:
sns.distplot(housing['population']);

In [51]:
housing["population"].where(housing["population"] > 2.5, 2.5, inplace=True)
sns.distplot(housing['population']);

<h5>dael with "total_rooms"

In [52]:
sns.distplot(housing['total_rooms']);

In [53]:
housing["total_rooms"] = np.log1p(housing["total_rooms"])
sns.distplot(housing['total_rooms']);

In [54]:
housing["total_rooms"].where(housing["total_rooms"] > 4, 4, inplace=True)
sns.distplot(housing['total_rooms']);

<h3>P-value / VIF check  - 다중공산성 Multicollinearity 확인 위한 VIF/ p-value 체크

<h5> P-value 체크

In [55]:
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [56]:
housing.columns

In [57]:
features = "longitude+latitude+housing_median_age+total_rooms+total_bedrooms+population+households+median_income+ocean_proximity+income_cat+rooms_per_household+bedrooms_per_room+population_per_household"


In [58]:
# Break into left and right hand side; y and X
y, X = dmatrices("median_house_value ~" +features, data=housing, return_type="dataframe")

# For each Xi, calculate VIF
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Fit X to y
result = sm.OLS(y, X).fit()

In [59]:
print(result.summary())

- We can drop few variables and select only those that have p values < 0.5 and then we can check improvement in the model. 
- total_bedrooms 칼럼 제거

In [60]:
housing.drop('total_bedrooms', axis=1, inplace=True)

<h5> VIF체크

In [61]:
features = "longitude+latitude+housing_median_age+total_rooms+population+households+median_income+ocean_proximity+income_cat+rooms_per_household+bedrooms_per_room+population_per_household"


In [62]:
#Break into left and right hand side; y and X
y, X = dmatrices("median_house_value ~" +features, data=housing, return_type="dataframe")

In [64]:
#For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

In [65]:
vif.round(1)

**In general, you can drop feature which over 10 vif values**<br>
-다중공산성 Multicollinearity 체크 --> 원래 10이 넘으면 문제가 있는것으로 보고 처리해줘야 한다

In [66]:
#you can drop column like this 
#housing.drop('households', axis=1, inplace=True)


In [67]:
#바뀐 데이터로 다시 train test 나눠준다 
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [68]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

--------------------------------------------------------------------

# 머신러닝 알고리즘에 적용하기 위해 데이터 준비 (Prepare the data for Machine Learning algorithms)   

책의 이 부분은 "분석 자동화"를 지향하여 전개됩니다. 

`strat_train_set` : 앞서 준비한 stratified train set (원 데이터의 20% 분량이었음) 

In [69]:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels(median_house_value) for training set
housing_labels = strat_train_set["median_house_value"].copy()

- housing : training set의 Predictor 들 
- housing_labels : training set의 target label 

### 데이터 클리닝 

앞서 우리는 `total_bedrooms` 열에 결측치(Missing Value)가 있는 것을 보았음.  이를 해결하자.
1. Get rid of the corresponding districts : 결측치가 있는 해당 row를 제거. 간단함. 데이터 사이즈가 작아짐 
2. Get rid of the whole attribute. : 아예 결측치가 있는 열을 제거. 별로 좋은 생각 아닐 수도 
3. Set the values to some value (zero, the mean, the median, etc.) : 결측치에 적절한 값을 넣어 사용. 

In [70]:
housing.info()

In [71]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows    # 모두 total_bedrooms 에 결측치가 있음  

Scikit-Learn의 Imputer 클래스 사용:

In [72]:
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy="median")  

Remove the text attribute because median can only be calculated on numerical attributes:

In [73]:
housing_num = housing.drop('ocean_proximity', axis=1)
# alternatively: housing_num = housing.select_dtypes(include=[np.number])

In [74]:
imputer.fit(housing_num)

In [75]:
imputer.statistics_

Check that this is the same as manually computing the median of each attribute:

In [76]:
housing_num.median().values

In [77]:
housing_num.median().values

Transform the training set: 여기서 training set은 결측치가 있는 원 데이터. 앞에서 `imputer` 가 median 값으로 결측치를 메꾸는 작업을 하는 것을 배웠으니, 이를 결측치가 있는 training set에 적용하면, 결측치에다 해당 median 값을 대치할 것임  

In [78]:
X = imputer.transform(housing_num)

In [79]:
X   # X는 numpy의 ndarray

In [80]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,   # 결측치가 대치된 X를 pandas DataFrame으로 변환 
                          index = list(housing.index.values))  

In [81]:
sample_incomplete_rows.index.values   

In [82]:
housing_tr.loc[sample_incomplete_rows.index.values]   # 확인 

In [83]:
imputer.strategy

In [84]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
housing_tr.head()

### Handling Text and Categorical Attributes
Now let's preprocess the categorical input feature, `ocean_proximity`: Scikit-Learn 의 대부분 predictor estimator들은 카테고리형 변수 (정성적 변수)를 숫자형으로 변환해 줘야 작동함 

In [85]:
housing_cat = housing[['ocean_proximity']]
housing_cat.head(10)

# labelbarazer사용

In [86]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
housing_cat_1hot = encoder.fit_transform(housing_cat)
housing_cat_1hot

Scikit-Learn 라이브러리가 계속 업데이트되면서 더 많은, 편리한 함수들이 만들어지고 있음.  

### Custom Transformers : 나에게 딱 맞는 데이터 조작 툴을 만들어, 한 번에 여러 데이터 조작을 하고, 사용하기도 쉽게 하고, 자동화도 되게 하자.  
Let's create a custom transformer to add extra attributes:

In [87]:
housing.info()

In [88]:
from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

"""
CombinedAttributesAdder :
앞에서 bedrooms, population attribute(column, 열) 들에 대해 행한 조작을 한 번에 처리하기 위한 클래스 
"""
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):  
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)


   
<br>
**Feature Scaling** : 앞에서 각 열들이 갖는 값의 범위가 매우 다름을 보았다. 열들이 취할 수 있는 값들의 range를 비슷하게 하자.   

Now let's build a pipeline for preprocessing the numerical attributes:

In [89]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', Imputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [90]:
housing_num.info()

In [91]:
housing_num_tr

In [92]:
housing.head()

#### A transformer to just select `a subset of the Pandas DataFrame columns` and return in ndarray:

In [93]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
 

num_pipeline = Pipeline([
        ('imputer', Imputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

#oc~만 뺴고 housing_num에 담는다
housing_num = housing.drop("ocean_proximity",axis=1)
housing_num

In [94]:
#이를 
housing_num_tr = num_pipeline.fit_transform(housing_num)

In [95]:
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [96]:
from sklearn.base import BaseEstimator, TransformerMixin

# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet

#credit to @hesenp 
class LabelBinarizerPipelineFriendly(LabelBinarizer):
    def fit(self, X, y=None):
        """this would allow us to fit the model based on the X input."""
        super(LabelBinarizerPipelineFriendly, self).fit(X)
    def transform(self, X, y=None):
        return super(LabelBinarizerPipelineFriendly, self).transform(X)

    def fit_transform(self, X, y=None):
        return super(LabelBinarizerPipelineFriendly, self).fit(X).transform(X)


class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values


In [97]:
from sklearn.pipeline import FeatureUnion

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('label_binarizer', LabelBinarizerPipelineFriendly()),
    ])

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline)
    ])

In [98]:
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

Now let's join all these components into a big pipeline that will preprocess both the numerical and the categorical features:

In [99]:
housing_prepared.shape

#### we can do real machine - learning from now on

In [100]:
pd.DataFrame(housing_prepared).info()

In [101]:
pd.DataFrame(housing_prepared).head()   # 모든 열들에 결측치 없고, standardize, 수치화 

-----------------------------------
# 적절한 머신러닝 모델을 선정해 학습    
(Select and train a a model )

<br>

### 선형 회귀 모델을 사용해 보자

In [102]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()    # 선형 회귀 알고리즘을 실행할 수 있는 객체(lin_reg)를 만들어 
#lin_reg = LinearRegression(normalize=True, copy_X=True, n_jobs=1)

# lin_reg 에게 입력 training set인 housing_prepared 와 해당 레이블인 housing_labels을 주며 
# 학습(fit) 하라고 지시.  학습된 모델은 lin_reg 자신에게 있음 
lin_reg.fit(X=housing_prepared, y=housing_labels)    

## 학습 끝.  학습된 모델 생성. 

<br>

학습할 데이터가 준비되니, 학습 모델을 선정하고 학습과정 자체는 일도 아님.  (그러나 보통 이렇게 간단하지는 않음).   
그럼, 이제 만들어진 모델이 과연 얼마나 일을 잘하나 평가해 보자.


In [103]:
# let's try the full pipeline on a few training instances
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", lin_reg.predict(some_data_prepared))   # 이것이 학습한 모델이 예측한 값 

Compare against the actual values:

In [104]:
print("Labels:", list(some_labels))      # 이것이 실제 값 (정답)

In [105]:
some_data_prepared

In [106]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(y_true=housing_labels, y_pred=housing_predictions)    # MSE
lin_rmse = np.sqrt(lin_mse)    # RMSE 
lin_rmse

Target value of mode(median_house_value)l is between 120,000 - 265,000

모델의 target 치가 120,000 과 265,000 사이에서 움직이는데, 





# Fine-tune your model
## 더 신뢰가는 평가 방법을 사용하자

In [109]:
from sklearn.model_selection import cross_val_score   # K-Fold Cross Validation 

 

In [110]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())



In [111]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

- Average of 10 fold validation is 66097<br>
